<a href="https://colab.research.google.com/github/MdEmonMiaOfficial/Fusion-of-Clinical-Metadata-and-3D-CT-Image-Features-Interpretable-Lung-Cancer-Classification/blob/main/Interpretable_Lung_Cancer_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import files
uploaded = files.upload()



import os, zipfile
ZIP_PATH = list(uploaded.keys())[0] if 'uploaded' in globals() else ZIP_PATH

EXTRACT_DIR = "/content/preproc"
os.makedirs(EXTRACT_DIR, exist_ok=True)
with zipfile.ZipFile(ZIP_PATH, 'r') as zf:
    zf.extractall(EXTRACT_DIR)


import os, numpy as np
files_npz = [f for f in os.listdir(EXTRACT_DIR) if f.endswith(".npz")]
print("Total NPZ:", len(files_npz))
d = np.load(os.path.join(EXTRACT_DIR, files_npz[0]))
print("Keys:", d.files, "x shape:", d["x"].shape, "y:", d["y"])


Saving preproc.zip to preproc.zip
Total NPZ: 95
Keys: ['x', 'y'] x shape: (1, 128, 128, 128) y: 1


In [ ]:
class NPZ3DDataset(Dataset):
    def __init__(self, folder):
        self.paths = sorted(glob.glob(os.path.join(folder, "*.npz")))
        assert len(self.paths)>0, "No .npz found!"
    def __len__(self): return len(self.paths)
    def __getitem__(self, idx):
        d = np.load(self.paths[idx])
        x = torch.from_numpy(d["x"]).float()         # (1, D, H, W) in [0,1]
        y = torch.tensor(int(d["y"]), dtype=torch.long)
        return x, y

ds = NPZ3DDataset(DATA_DIR)
labels = [int(np.load(p)['y']) for p in ds.paths]
print("Total:", len(ds), "| class0:", labels.count(0), "| class1:", labels.count(1))

# stratified 70/15/15 split
idx0 = [i for i,y in enumerate(labels) if y==0]
idx1 = [i for i,y in enumerate(labels) if y==1]
def split_idx(idxs, frac_tr=0.7, frac_va=0.15):
    idxs = idxs[:]; random.Random(SEED).shuffle(idxs)
    n=len(idxs); ntr=int(n*frac_tr); nva=int(n*frac_va)
    return idxs[:ntr], idxs[ntr:ntr+nva], idxs[ntr+nva:]
tr0,va0,te0 = split_idx(idx0); tr1,va1,te1 = split_idx(idx1)

train_idx = tr0 + tr1; val_idx = va0 + va1; test_idx = te0 + te1
random.shuffle(train_idx); random.shuffle(val_idx); random.shuffle(test_idx)
print(f"Split → train={len(train_idx)} val={len(val_idx)} test={len(test_idx)}")


Total: 95 | class0: 48 | class1: 47
Split → train=65 val=14 test=16


In [ ]:
# balanced sampler (train)
train_labels = [labels[i] for i in train_idx]
class_count = np.array([train_labels.count(0), train_labels.count(1)])
w = 1.0 / np.where(class_count==0, 1, class_count)
sample_weights = np.array([w[l] for l in train_labels])
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

train_dl = DataLoader(Subset(ds, train_idx), batch_size=BATCH_SIZE, sampler=sampler, num_workers=2, pin_memory=True)
val_dl   = DataLoader(Subset(ds, val_idx),   batch_size=BATCH_SIZE, shuffle=False)
test_dl  = DataLoader(Subset(ds, test_idx),  batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
from monai.networks.nets import DenseNet121

model = DenseNet121(
    spatial_dims=3,
    in_channels=1,
    out_channels=2
).to(device)


In [ ]:
# ---------- লস/অপ্টিম/স্কেজিউলার ----------
n0, n1 = labels.count(0), labels.count(1)
w0 = 1.0 if n0==0 else (len(labels)/(2.0*n0))
w1 = 1.0 if n1==0 else (len(labels)/(2.0*n1))
class_weights = torch.tensor([w0, w1], device=device, dtype=torch.float32)

crit = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.05)
opt  = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', factor=0.5, patience=4)
scaler = GradScaler(enabled=torch.cuda.is_available())


/tmp/ipython-input-2756800604.py:10: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=torch.cuda.is_available())


In [ ]:
# ---------- অগমেন্টেশন ----------
def augment3d(x):  # x: (B,1,D,H,W)
    import random, torch
    if random.random()<0.5: x = torch.flip(x, [2])  # Z flip
    if random.random()<0.5: x = torch.flip(x, [3])  # Y flip
    if random.random()<0.5: x = torch.flip(x, [4])  # X flip
    if random.random()<0.3: x = x.transpose(2,3)    # 90° Z<->Y
    if random.random()<0.3: x = x.transpose(3,4)    # 90° Y<->X

In [ ]:
# ---------- ট্রেন/ভ্যাল লুপ ----------
def run_epoch(dl, train=True):
    model.train(train)
    total = correct = 0
    loss_sum = 0.0
    for x,y in dl:
        if train:
            x = augment3d(x)
            opt.zero_grad(set_to_none=True)
        x, y = x.to(device), y.to(device)
        with autocast(enabled=torch.cuda.is_available()):
            logits = model(x)
            loss   = crit(logits, y)
        if train:
            scaler.scale(loss).backward()
            scaler.step(opt)
            scaler.update()
        loss_sum += float(loss.detach().item()) * x.size(0)
        correct  += (logits.argmax(1)==y).sum().item()
        total    += x.size(0)
    return loss_sum/max(1,total), correct/max(1,total)

In [ ]:
import torch, random

# 0) CPU হলে pin_memory=False করে দিন
PIN_MEMORY = torch.cuda.is_available()

# (ঐচ্ছিক) যদি DataLoader আগে বানানো থাকে, নতুন করে বানান
# train_dl = DataLoader(Subset(ds, train_idx), batch_size=BATCH_SIZE,
#                       sampler=sampler, num_workers=2, pin_memory=PIN_MEMORY)
# val_dl   = DataLoader(Subset(ds, val_idx),   batch_size=BATCH_SIZE,
#                       shuffle=False, num_workers=2, pin_memory=PIN_MEMORY)
# test_dl  = DataLoader(Subset(ds, test_idx),  batch_size=BATCH_SIZE,
#                       shuffle=False, num_workers=2, pin_memory=PIN_MEMORY)

# 1) স্যানিটি-চেক: এক ব্যাচ এনে টাইপ/শেপ দেখুন
xb, yb = next(iter(train_dl))
print("Sample batch types:", type(xb), type(yb))
print("Sample batch shapes:", xb.shape, yb.shape)
assert xb is not None and yb is not None, "❌ DataLoader returned None"
assert isinstance(xb, torch.Tensor) and isinstance(yb, torch.Tensor), "❌ Not tensors"

# 2) সেফ অগমেন্টেশন: সব পথে নিশ্চিতভাবে `x` ফেরত দেয়
def augment3d(x: torch.Tensor) -> torch.Tensor:
    # ইনপুট চেক
    if not isinstance(x, torch.Tensor):
        raise TypeError("augment3d expects a torch.Tensor")
    if x.ndim != 5:
        raise ValueError(f"Expected (B, C, D, H, W). Got shape={tuple(x.shape)}")

    # ফ্লিপ
    if random.random() < 0.5: x = torch.flip(x, [2])  # D
    if random.random() < 0.5: x = torch.flip(x, [3])  # H
    if random.random() < 0.5: x = torch.flip(x, [4])  # W
    # 90° রোটেশন (axis swap)
    if random.random() < 0.3: x = x.transpose(2,3)    # D<->H
    if random.random() < 0.3: x = x.transpose(3,4)    # H<->W

    # কনটিগুয়াস টেনসর (কিছু মডিউল non-contiguous পছন্দ করে না)
    return x.contiguous()

# 3) সঠিক run_epoch: সবসময় (avg_loss, acc) রিটার্ন করে
from torch.cuda.amp import autocast
# PyTorch >=2.0 হলে:
scaler = torch.amp.GradScaler("cuda", enabled=torch.cuda.is_available())

def run_epoch(dl, train: bool = True):
    model.train(train)
    total = 0
    correct = 0
    loss_sum = 0.0

    for x, y in dl:
        # স্যানিটি
        if x is None or y is None:
            raise RuntimeError("Got None from DataLoader; check augment/run_epoch definitions.")
        if train:
            x = augment3d(x)
            opt.zero_grad(set_to_none=True)

        x, y = x.to(device), y.to(device)

        with autocast(device_type="cuda", enabled=torch.cuda.is_available()):
            logits = model(x)
            loss = crit(logits, y)

        if train:
            scaler.scale(loss).backward()
            scaler.step(opt)
            scaler.update()

        loss_sum += float(loss.detach().item()) * x.size(0)
        preds = logits.argmax(1)
        correct += (preds == y).sum().item()
        total += x.size(0)

    avg_loss = loss_sum / max(total, 1)
    acc = correct / max(total, 1)
    return avg_loss, acc

print("✅ augment3d & run_epoch redefined safely.")


Sample batch types: <class 'torch.Tensor'> <class 'torch.Tensor'>
Sample batch shapes: torch.Size([2, 1, 128, 128, 128]) torch.Size([2])
✅ augment3d & run_epoch redefined safely.


In [ ]:
# ==== Lightweight 3D CNN Training Script ====
import os, glob, random, numpy as np, torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset, WeightedRandomSampler

# ---------- Config ----------
DATA_DIR     = "/content/preproc"   # আপনার .npz ফোল্ডার
BATCH_SIZE   = 2                    # GPU কম হলে 1 দিন
EPOCHS       = 60                   # early stopping আছে
LR           = 1e-4
WEIGHT_DECAY = 1e-4
SEED         = 42
PATIENCE     = 10
CKPT         = "/content/best_light3dcnn.pt"

random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
device = "cuda" if torch.cuda.is_available() else "cpu"
PIN_MEMORY = torch.cuda.is_available()
print("Device:", device)

# ---------- Dataset ----------
class NPZ3DDataset(Dataset):
    def __init__(self, folder):
        self.paths = sorted(glob.glob(os.path.join(folder, "*.npz")))
        assert len(self.paths)>0, "No .npz found in DATA_DIR!"
    def __len__(self): return len(self.paths)
    def __getitem__(self, idx):
        d = np.load(self.paths[idx])
        x = torch.from_numpy(d["x"]).float()         # (1, D, H, W) in [0,1]
        y = torch.tensor(int(d["y"]), dtype=torch.long)
        return x, y

ds = NPZ3DDataset(DATA_DIR)
labels = [int(np.load(p)["y"]) for p in ds.paths]
print("Total:", len(ds), "| class0:", labels.count(0), "| class1:", labels.count(1))

# ---------- Stratified split ----------
idx0 = [i for i,y in enumerate(labels) if y==0]
idx1 = [i for i,y in enumerate(labels) if y==1]
def split_idx(idxs, frac_tr=0.7, frac_va=0.15):
    idxs = idxs[:]; random.Random(SEED).shuffle(idxs)
    n=len(idxs); ntr=int(n*frac_tr); nva=int(n*frac_va)
    return idxs[:ntr], idxs[ntr:ntr+nva], idxs[ntr+nva:]
tr0,va0,te0 = split_idx(idx0); tr1,va1,te1 = split_idx(idx1)
train_idx = tr0+tr1; val_idx = va0+va1; test_idx = te0+te1
random.shuffle(train_idx); random.shuffle(val_idx); random.shuffle(test_idx)
print(f"Split → train={len(train_idx)}  val={len(val_idx)}  test={len(test_idx)}")

# ---------- Loaders (balanced sampler for train) ----------
train_labels = [labels[i] for i in train_idx]
class_count  = np.array([train_labels.count(0), train_labels.count(1)])
w = 1.0 / np.where(class_count==0, 1, class_count)
sample_weights = np.array([w[l] for l in train_labels], dtype=np.float64)
sampler   = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

train_dl = DataLoader(Subset(ds, train_idx), batch_size=BATCH_SIZE, sampler=sampler,
                      num_workers=2, pin_memory=PIN_MEMORY)
val_dl   = DataLoader(Subset(ds, val_idx),   batch_size=BATCH_SIZE, shuffle=False,
                      num_workers=2, pin_memory=PIN_MEMORY)
test_dl  = DataLoader(Subset(ds, test_idx),  batch_size=BATCH_SIZE, shuffle=False,
                      num_workers=2, pin_memory=PIN_MEMORY)

# ---------- Lightweight 3D CNN ----------
class Light3DCNN(nn.Module):
    def __init__(self, n_classes=2):
        super().__init__()
        def block(cin, cout):
            return nn.Sequential(
                nn.Conv3d(cin, cout, 3, padding=1),
                nn.BatchNorm3d(cout),
                nn.ReLU(inplace=True),
                nn.Conv3d(cout, cout, 3, padding=1),
                nn.BatchNorm3d(cout),
                nn.ReLU(inplace=True),
                nn.MaxPool3d(2)
            )
        self.features = nn.Sequential(
            block(1, 16),   # /2
            block(16, 32),  # /4
            block(32, 64),  # /8
        )
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool3d(1),
            nn.Flatten(),
            nn.Linear(64, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(128, n_classes)
        )
    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

model = Light3DCNN().to(device)

# ---------- Loss / Optim / Scheduler ----------
n0, n1 = labels.count(0), labels.count(1)
w0 = 1.0 if n0==0 else (len(labels)/(2.0*n0))
w1 = 1.0 if n1==0 else (len(labels)/(2.0*n1))
class_weights = torch.tensor([w0, w1], device=device, dtype=torch.float32)

crit = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.05)
opt  = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', factor=0.5, patience=4)

scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

# ---------- Augmentation ----------
def augment3d(x: torch.Tensor) -> torch.Tensor:
    if random.random()<0.5: x = torch.flip(x, [2])  # D
    if random.random()<0.5: x = torch.flip(x, [3])  # H
    if random.random()<0.5: x = torch.flip(x, [4])  # W
    if random.random()<0.3: x = x.transpose(2,3)
    if random.random()<0.3: x = x.transpose(3,4)
    return x.contiguous()

# ---------- Train/Val loop ----------
def run_epoch(dl, train=True):
    model.train(train)
    total = correct = 0
    loss_sum = 0.0
    for x,y in dl:
        if train:
            x = augment3d(x)
            opt.zero_grad(set_to_none=True)
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            logits = model(x)
            loss   = crit(logits, y)

        if train:
            scaler.scale(loss).backward()
            scaler.step(opt)
            scaler.update()

        loss_sum += float(loss.detach().item()) * x.size(0)
        correct  += (logits.argmax(1)==y).sum().item()
        total    += x.size(0)

    return loss_sum/max(1,total), correct/max(1,total)

# ---------- Training ----------
best_val_acc, best_val_loss, patience = 0.0, float("inf"), 0
for epoch in range(1, EPOCHS+1):
    tr_loss, tr_acc = run_epoch(train_dl, True)
    va_loss, va_acc = run_epoch(val_dl, False)

    scheduler.step(va_loss)

    improved = (va_acc > best_val_acc) or (va_acc == best_val_acc and va_loss < best_val_loss)
    if improved:
        best_val_acc, best_val_loss = va_acc, va_loss
        torch.save(model.state_dict(), CKPT)
        patience = 0
    else:
        patience += 1

    print(f"Epoch {epoch:02d} | train {tr_loss:.4f}/{tr_acc:.3f} "
          f"| val {va_loss:.4f}/{va_acc:.3f} | best_val_acc {best_val_acc:.3f}")

    if patience >= PATIENCE:
        print("Early stopping.")
        break

# ---------- Test ----------
model.load_state_dict(torch.load(CKPT, map_location=device))
model.eval()
all_y, all_p = [], []
with torch.no_grad():
    for x,y in test_dl:
        x = x.to(device)
        pred = model(x).argmax(1).cpu().numpy()
        all_p.extend(pred.tolist()); all_y.extend(y.numpy().tolist())

all_y = np.array(all_y); all_p = np.array(all_p)
acc = (all_y == all_p).mean()
cm = np.zeros((2,2), dtype=int)  # [[TN FP],[FN TP]]
for yt, yp in zip(all_y, all_p): cm[yt, yp] += 1

print(f"\nTEST ACCURACY: {acc:.3f}")
print("Confusion Matrix [[TN FP],[FN TP]]:\n", cm)
print("Unique predictions:", sorted(set(all_p)))


Device: cuda
Total: 95 | class0: 48 | class1: 47
Split → train=65  val=14  test=16


/tmp/ipython-input-2410657665.py:105: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
/tmp/ipython-input-2410657665.py:127: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):


Epoch 01 | train 0.7434/0.492 | val 0.6917/0.571 | best_val_acc 0.571
Epoch 02 | train 0.6995/0.477 | val 0.7132/0.500 | best_val_acc 0.571
Epoch 03 | train 0.7055/0.446 | val 0.7105/0.500 | best_val_acc 0.571
Epoch 04 | train 0.7088/0.446 | val 0.6822/0.571 | best_val_acc 0.571
Epoch 05 | train 0.7085/0.462 | val 0.6690/0.500 | best_val_acc 0.571
Epoch 06 | train 0.6803/0.569 | val 0.6687/0.500 | best_val_acc 0.571
Epoch 07 | train 0.6883/0.569 | val 0.6605/0.571 | best_val_acc 0.571
Epoch 08 | train 0.6938/0.554 | val 0.6608/0.571 | best_val_acc 0.571
Epoch 09 | train 0.7152/0.462 | val 0.6694/0.500 | best_val_acc 0.571
Epoch 10 | train 0.6873/0.585 | val 0.6633/0.714 | best_val_acc 0.714
Epoch 11 | train 0.6686/0.585 | val 0.6771/0.500 | best_val_acc 0.714
Epoch 12 | train 0.7079/0.538 | val 0.6615/0.571 | best_val_acc 0.714
Epoch 13 | train 0.6879/0.492 | val 0.6633/0.714 | best_val_acc 0.714
Epoch 14 | train 0.6761/0.631 | val 0.6720/0.714 | best_val_acc 0.714
Epoch 15 | train 0.7

SyntaxError: invalid syntax (ipython-input-1823377705.py, line 13)

In [ ]:
META_CSV = "/content/metadata.csv"  # আপনার আপলোড করা ফাইলের নাম/পাথ
import os
print("Found?" , os.path.exists(META_CSV))

Found? True


In [ ]:
meta = pd.read_csv(META_CSV)
print(meta.head())


   patient_id  label  age smoking_status
0          10      1   74         former
1          28      1   47         former
2           1      0   42         smoker
3          53      1   51         smoker
4          38      0   58         smoker


In [ ]:
# ====== Image (3D CNN 64D) + Metadata Fusion → Random Forest ======
!pip -q install scikit-learn

import os, glob, random, numpy as np, pandas as pd, torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

# ---------------- Config ----------------
DATA_DIR   = "/content/preproc"          # আপনার npz dataset folder
META_CSV   = "/content/metadata.csv"     # আপনার uploaded metadata file
BATCH_SIZE = 2
EPOCHS_ENC = 20
LR         = 1e-4
SEED       = 42
device = "cuda" if torch.cuda.is_available() else "cpu"
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
print("Device:", device)

# ---------------- Dataset ----------------
class NPZ3D(Dataset):
    def __init__(self, folder):
        self.paths = sorted(glob.glob(os.path.join(folder, "*.npz")))
        assert self.paths, "No .npz found!"
    def __len__(self): return len(self.paths)
    def __getitem__(self, idx):
        p = self.paths[idx]; d = np.load(p)
        x = torch.from_numpy(d["x"]).float()
        y = torch.tensor(int(d["y"]), dtype=torch.long)
        stem = os.path.splitext(os.path.basename(p))[0]
        try: npz_id = int(stem)  # e.g. 0010.npz -> 10
        except: npz_id = stem
        return x, y, npz_id

full_ds = NPZ3D(DATA_DIR)
labels = [int(np.load(p)['y']) for p in full_ds.paths]
print(f"Total {len(full_ds)} | class0={labels.count(0)} class1={labels.count(1)}")

# ---------------- Split ----------------
idx0=[i for i,y in enumerate(labels) if y==0]
idx1=[i for i,y in enumerate(labels) if y==1]
def split(idxs):
    random.Random(SEED).shuffle(idxs); n=len(idxs)
    return idxs[:int(0.7*n)], idxs[int(0.7*n):int(0.85*n)], idxs[int(0.85*n):]
tr0,va0,te0=split(idx0); tr1,va1,te1=split(idx1)
train_idx=tr0+tr1; val_idx=va0+va1; test_idx=te0+te1

def subset(ds, idxs):
    class _S(Dataset):
        def __init__(self, base, sel): self.base, self.sel=base, sel
        def __len__(self): return len(self.sel)
        def __getitem__(self,i): return self.base[self.sel[i]]
    return _S(ds, idxs)

train_dl = DataLoader(subset(full_ds,train_idx), batch_size=BATCH_SIZE, shuffle=True)
val_dl   = DataLoader(subset(full_ds,val_idx),   batch_size=BATCH_SIZE, shuffle=False)
test_dl  = DataLoader(subset(full_ds,test_idx),  batch_size=BATCH_SIZE, shuffle=False)

# ---------------- Light 3D Encoder ----------------
class Light3DEncoder(nn.Module):
    def __init__(self, out_ch=64):
        super().__init__()
        def blk(cin,cout):
            return nn.Sequential(
                nn.Conv3d(cin, cout, 3,padding=1), nn.BatchNorm3d(cout), nn.ReLU(),
                nn.Conv3d(cout, cout,3,padding=1), nn.BatchNorm3d(cout), nn.ReLU(),
                nn.MaxPool3d(2))
        self.body=nn.Sequential(blk(1,16), blk(16,32), blk(32,out_ch))
        self.pool=nn.AdaptiveAvgPool3d(1)
        self.fc  = nn.Linear(out_ch,2)
    def forward(self,x,feat_only=False):
        z=self.body(x); z=self.pool(z).flatten(1)
        if feat_only: return z
        return self.fc(z)

enc = Light3DEncoder(64).to(device)
opt = torch.optim.AdamW(enc.parameters(), lr=LR)
crit= nn.CrossEntropyLoss()

def run_epoch(dl, train=True):
    enc.train(train)
    tot=0; corr=0; loss_sum=0.0
    for x,y,_ in dl:
        if train: opt.zero_grad(set_to_none=True)
        x,y=x.to(device), y.to(device)
        logits=enc(x)
        loss=crit(logits,y)
        if train: loss.backward(); opt.step()
        loss_sum+=float(loss.item())*x.size(0)
        corr+=(logits.argmax(1)==y).sum().item(); tot+=x.size(0)
    return loss_sum/max(1,tot), corr/max(1,tot)

print(">> Warm-up encoder...")
best_va=0; pat=0
for ep in range(1,EPOCHS_ENC+1):
    tr_loss,tr_acc=run_epoch(train_dl,True)
    va_loss,va_acc=run_epoch(val_dl,False)
    if va_acc>best_va:
        best_va=va_acc; pat=0
        torch.save(enc.state_dict(),"/content/enc_best.pt")
    else:
        pat+=1
    print(f"Ep{ep:02d} | tr {tr_loss:.3f}/{tr_acc:.3f} | va {va_loss:.3f}/{va_acc:.3f} | best {best_va:.3f}")
    if pat>=5: break

enc.load_state_dict(torch.load("/content/enc_best.pt", map_location=device))

# ---------------- Extract Features ----------------
def extract(dl):
    enc.eval(); X=[]; Y=[]; IDs=[]
    with torch.no_grad():
        for x,y,i in dl:
            x=x.to(device); z=enc(x,feat_only=True)
            X.append(z.cpu().numpy()); Y+=y.numpy().tolist(); IDs+=list(i)
    return np.concatenate(X,0), np.array(Y), np.array(IDs)

Xtr,ytr,idtr=extract(train_dl); Xva,yva,idva=extract(val_dl); Xte,yte,idte=extract(test_dl)
print("Image feature shape:", Xtr.shape)

# ---------------- Metadata Processing ----------------
meta = pd.read_csv(META_CSV)
assert "patient_id" in meta.columns, "metadata.csv must have patient_id column"

# age clean
if "age" in meta.columns:
    def parse_age(v):
        try: return float("".join(ch for ch in str(v) if ch.isdigit()))
        except: return np.nan
    meta["age"] = meta["age"].apply(parse_age)

num_cols = [c for c in ["age"] if c in meta.columns]
cat_cols = [c for c in ["smoking_status","gender","family_history"] if c in meta.columns]

def make_meta_matrix(ids):
    rows=[]
    for i in ids:
        row = meta[meta["patient_id"]==i]
        if row.empty:
            rows.append(np.zeros(len(num_cols)+len(cat_cols)))
        else:
            vals=[]
            if num_cols:
                vals += row[num_cols].iloc[0].astype(float).fillna(0).values.tolist()
            if cat_cols:
                vals += row[cat_cols].iloc[0].astype(str).values.tolist()
            rows.append(vals)
    df = pd.DataFrame(rows, columns=num_cols+cat_cols)
    if cat_cols:
        for c in cat_cols: df[c]=df[c].astype(str).fillna("UNK")
        oh = pd.get_dummies(df[cat_cols])
        df = pd.concat([df[num_cols], oh], axis=1)
    else:
        df = df[num_cols]
    return df.fillna(0).to_numpy(np.float32)

Xtr_meta=make_meta_matrix(idtr)
Xva_meta=make_meta_matrix(idva)
Xte_meta=make_meta_matrix(idte)
print("Meta dims:", Xtr_meta.shape[1])

# ---------------- Concatenate ----------------
Xtr_f=np.concatenate([Xtr,Xtr_meta],1)
Xva_f=np.concatenate([Xva,Xva_meta],1)
Xte_f=np.concatenate([Xte,Xte_meta],1)
print("Final feature dims:", Xtr_f.shape[1])

# ---------------- Random Forest ----------------
rf=RandomForestClassifier(n_estimators=400, class_weight="balanced", random_state=SEED)
rf.fit(np.vstack([Xtr_f,Xva_f]), np.hstack([ytr,yva]))

pred=rf.predict(Xte_f)
acc=accuracy_score(yte,pred); cm=confusion_matrix(yte,pred); f1=f1_score(yte,pred)
print(f"\nTEST ACC: {acc:.3f} | F1={f1:.3f}")
print("CM [[TN FP],[FN TP]]:\n",cm)


Device: cuda
Total 95 | class0=48 class1=47
>> Warm-up encoder...
Ep01 | tr 0.753/0.477 | va 0.739/0.500 | best 0.500
Ep02 | tr 0.716/0.477 | va 0.673/0.500 | best 0.500
Ep03 | tr 0.699/0.600 | va 0.706/0.357 | best 0.500
Ep04 | tr 0.702/0.523 | va 0.688/0.500 | best 0.500
Ep05 | tr 0.694/0.554 | va 0.702/0.500 | best 0.500
Ep06 | tr 0.684/0.600 | va 0.664/0.571 | best 0.571
Ep07 | tr 0.672/0.600 | va 0.663/0.500 | best 0.571
Ep08 | tr 0.687/0.631 | va 0.673/0.500 | best 0.571
Ep09 | tr 0.693/0.569 | va 0.674/0.500 | best 0.571
Ep10 | tr 0.678/0.615 | va 0.651/0.500 | best 0.571
Ep11 | tr 0.695/0.554 | va 0.702/0.500 | best 0.571
Image feature shape: (65, 64)
Meta dims: 4
Final feature dims: 68

TEST ACC: 0.562 | F1=0.533
CM [[TN FP],[FN TP]]:
 [[5 3]
 [4 4]]


In [ ]:
# ===== Final Fusion Pipeline: 3D image (64D) + metadata → RF / LogReg / SVM =====
!pip -q install scikit-learn torchvision

import os, glob, random, numpy as np, pandas as pd, torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, classification_report

# ---------------- Config ----------------
DATA_DIR   = "/content/preproc"          # .npz folder (x: (1,D,H,W), y: 0/1)
META_CSV   = "/content/metadata.csv"     # metadata file (must have 'patient_id')
BATCH_SIZE = 2
EPOCHS_ENC = 35                          # encoder warm-up epochs (আগে 20 ছিল)
LR         = 1e-4
SEED       = 42
device = "cuda" if torch.cuda.is_available() else "cpu"
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
print("Device:", device)

# ---------------- Dataset ----------------
class NPZ3D(Dataset):
    def __init__(self, folder):
        self.paths = sorted(glob.glob(os.path.join(folder, "*.npz")))
        assert self.paths, f"No .npz found in {folder}"
    def __len__(self): return len(self.paths)
    def __getitem__(self, idx):
        p = self.paths[idx]
        d = np.load(p)
        x = torch.from_numpy(d["x"]).float()
        y = torch.tensor(int(d["y"]), dtype=torch.long)
        stem = os.path.splitext(os.path.basename(p))[0]
        try: npz_id = int(stem)       # e.g. 00012.npz → 12
        except: npz_id = stem
        return x, y, npz_id

full_ds = NPZ3D(DATA_DIR)
labels = [int(np.load(p)['y']) for p in full_ds.paths]
print(f"Total {len(full_ds)} | class0={labels.count(0)} class1={labels.count(1)}")

# ---------------- Stratified split (70/15/15) ----------------
idx0=[i for i,y in enumerate(labels) if y==0]
idx1=[i for i,y in enumerate(labels) if y==1]
def split(idxs):
    idxs = idxs[:]; random.Random(SEED).shuffle(idxs); n=len(idxs)
    return idxs[:int(0.7*n)], idxs[int(0.7*n):int(0.85*n)], idxs[int(0.85*n):]
tr0,va0,te0=split(idx0); tr1,va1,te1=split(idx1)
train_idx=tr0+tr1; val_idx=va0+va1; test_idx=te0+te1

def subset(ds, idxs):
    class _S(Dataset):
        def __init__(self, base, sel): self.base, self.sel=base, sel
        def __len__(self): return len(self.sel)
        def __getitem__(self,i): return self.base[self.sel[i]]
    return _S(ds, idxs)

train_dl = DataLoader(subset(full_ds,train_idx), batch_size=BATCH_SIZE, shuffle=True,  num_workers=2, pin_memory=torch.cuda.is_available())
val_dl   = DataLoader(subset(full_ds,val_idx),   batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=torch.cuda.is_available())
test_dl  = DataLoader(subset(full_ds,test_idx),  batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=torch.cuda.is_available())

# ---------------- 3D Encoder (Light) + Augmentation ----------------
class Light3DEncoder(nn.Module):
    def __init__(self, out_ch=64):
        super().__init__()
        def blk(cin, cout):
            return nn.Sequential(
                nn.Conv3d(cin, cout, 3, padding=1), nn.BatchNorm3d(cout), nn.ReLU(inplace=True),
                nn.Conv3d(cout, cout, 3, padding=1), nn.BatchNorm3d(cout), nn.ReLU(inplace=True),
                nn.MaxPool3d(2)
            )
        self.body = nn.Sequential(blk(1,16), blk(16,32), blk(32,out_ch))  # /2,/4,/8
        self.pool = nn.AdaptiveAvgPool3d(1)
        self.fc   = nn.Linear(out_ch, 2)   # warm-up head
    def forward(self, x, feat_only=False):
        z = self.body(x)
        z = self.pool(z).flatten(1)        # (B, out_ch)
        if feat_only: return z
        return self.fc(z)

def augment3d_light(x: torch.Tensor) -> torch.Tensor:
    # simple flips + axis swap + light noise
    if random.random()<0.5: x = torch.flip(x, [2])  # D
    if random.random()<0.5: x = torch.flip(x, [3])  # H
    if random.random()<0.5: x = torch.flip(x, [4])  # W
    if random.random()<0.3: x = x.transpose(2,3)
    if random.random()<0.3: x = x.transpose(3,4)
    if random.random()<0.5:
        noise = torch.randn_like(x)*0.02
        x = (x + noise).clamp(0,1)
    return x.contiguous()

enc = Light3DEncoder(64).to(device)
opt = torch.optim.AdamW(enc.parameters(), lr=LR, weight_decay=1e-4)
crit= nn.CrossEntropyLoss()

def run_epoch(dl, train=True):
    enc.train(train)
    tot=corr=0; loss_sum=0.0
    for x,y,_ in dl:
        if train:
            x = augment3d_light(x)
            opt.zero_grad(set_to_none=True)
        x,y = x.to(device), y.to(device)
        logits = enc(x)
        loss   = crit(logits, y)
        if train:
            loss.backward(); opt.step()
        loss_sum += float(loss.item()) * x.size(0)
        corr     += (logits.argmax(1)==y).sum().item()
        tot      += x.size(0)
    return loss_sum/max(1,tot), corr/max(1,tot)

print(">> Warm-up encoder…")
best_va=0.0; pat=0; PATIENCE=6
for ep in range(1, EPOCHS_ENC+1):
    tr_loss,tr_acc = run_epoch(train_dl, True)
    va_loss,va_acc = run_epoch(val_dl,   False)
    if va_acc>best_va:
        best_va=va_acc; pat=0
        torch.save(enc.state_dict(), "/content/enc_best.pt")
    else:
        pat+=1
    print(f"Ep{ep:02d} | tr {tr_loss:.3f}/{tr_acc:.3f} | va {va_loss:.3f}/{va_acc:.3f} | best {best_va:.3f}")
    if pat>=PATIENCE:
        print("Early stop."); break

enc.load_state_dict(torch.load("/content/enc_best.pt", map_location=device))

# ---------------- Extract 64D image features ----------------
def extract(dl):
    enc.eval(); X=[]; Y=[]; IDs=[]
    with torch.no_grad():
        for x,y,i in dl:
            x = x.to(device)
            z = enc(x, feat_only=True)     # (B,64)
            X.append(z.cpu().numpy()); Y+=y.numpy().tolist(); IDs+=list(i)
    return np.concatenate(X,0), np.array(Y), np.array(IDs)

Xtr_img,ytr,idtr = extract(train_dl)
Xva_img,yva,idva = extract(val_dl)
Xte_img,yte,idte = extract(test_dl)
print("Image feature shapes:", Xtr_img.shape, Xva_img.shape, Xte_img.shape)

# ---------------- Metadata: read + clean + one-hot ----------------
meta = pd.read_csv(META_CSV)
assert "patient_id" in meta.columns, "metadata.csv must have 'patient_id' column"

# helper to intify id if possible
def to_int_maybe(v):
    try: return int(str(v))
    except: return np.nan
meta["_pid_int"] = meta["patient_id"].apply(to_int_maybe)
if "age" in meta.columns:
    def parse_age(v):
        s = str(v); dig = "".join(ch for ch in s if ch.isdigit())
        return float(dig) if dig else np.nan
    meta["age"] = meta["age"].apply(parse_age)

num_cols = [c for c in ["age"] if c in meta.columns]
cat_cols = [c for c in ["smoking_status","gender","family_history"] if c in meta.columns]

def rows_for_ids(ids):
    rows=[]
    for rid in ids:
        r = meta[meta["_pid_int"]==rid]
        if r.empty:
            # fallback exact string match
            r = meta[ meta["patient_id"].astype(str) == str(rid) ]
        rows.append(r.iloc[0:1].copy() if not r.empty else pd.DataFrame(columns=meta.columns))
    return rows

def make_split_df(ids):
    rows = rows_for_ids(ids)
    out  = []
    for r in rows:
        if r.empty:
            rec = {}
        else:
            rec = r.iloc[0].to_dict()
        out.append(rec)
    df = pd.DataFrame(out)
    # keep only selected cols
    keep = []
    if num_cols: keep += num_cols
    if cat_cols: keep += cat_cols
    df = df[keep] if keep else pd.DataFrame(index=df.index)
    # defaults
    for c in num_cols: df[c] = pd.to_numeric(df.get(c, np.nan), errors="coerce")
    for c in cat_cols: df[c] = df.get(c, "UNK").astype(str)
    return df

df_tr = make_split_df(idtr)
df_va = make_split_df(idva)
df_te = make_split_df(idte)
full  = pd.concat([df_tr, df_va, df_te], axis=0, ignore_index=True)

# one-hot for cats; numeric fillna=median (simple)
if num_cols:
    for c in num_cols:
        med = full[c].median(skipna=True) if c in full.columns else 0.0
        full[c] = pd.to_numeric(full.get(c, med), errors="coerce").fillna(med)
num_part = full[num_cols] if num_cols else pd.DataFrame(index=full.index)
if cat_cols:
    for c in cat_cols:
        full[c] = full[c].fillna("UNK").astype(str)
    oh = pd.get_dummies(full[cat_cols], drop_first=False)
else:
    oh = pd.DataFrame(index=full.index)

fullX = pd.concat([num_part, oh], axis=1).astype(np.float32)
n_tr, n_va = len(df_tr), len(df_va)
Xtr_meta = fullX.iloc[:n_tr].to_numpy()
Xva_meta = fullX.iloc[n_tr:n_tr+n_va].to_numpy()
Xte_meta = fullX.iloc[n_tr+n_va:].to_numpy()
print("Meta dims:", Xtr_meta.shape[1])

# ---------------- Fusion features ----------------
Xtr_f = np.concatenate([Xtr_img, Xtr_meta], axis=1)
Xva_f = np.concatenate([Xva_img, Xva_meta], axis=1)
Xte_f = np.concatenate([Xte_img, Xte_meta], axis=1)
print("Final feature dims:", Xtr_f.shape[1])

# ---------------- RF (train->val threshold tune -> retrain on train+val) ----------------
rf_params = dict(
    n_estimators=800,
    max_depth=12,
    min_samples_leaf=2,
    class_weight="balanced",
    random_state=SEED,
    n_jobs=-1
)

# 1) fit on train only → tune threshold on val
rf1 = RandomForestClassifier(**rf_params)
rf1.fit(Xtr_f, ytr)
val_proba = rf1.predict_proba(Xva_f)[:,1]
best_t, best_f1 = 0.5, -1.0
for t in np.linspace(0.2, 0.8, 49):
    pred = (val_proba >= t).astype(int)
    f1   = f1_score(yva, pred)
    if f1 > best_f1:
        best_f1, best_t = f1, t
print(f"[RF] Best VAL threshold: {best_t:.3f} (F1={best_f1:.3f})")

# 2) retrain on train+val
Xtrva_f = np.vstack([Xtr_f, Xva_f])
ytrva   = np.hstack([ytr, yva])
rf = RandomForestClassifier(**rf_params)
rf.fit(Xtrva_f, ytrva)

# 3) test with tuned threshold
test_proba = rf.predict_proba(Xte_f)[:,1]
rf_pred = (test_proba >= best_t).astype(int)
rf_acc = accuracy_score(yte, rf_pred); rf_f1 = f1_score(yte, rf_pred); rf_cm = confusion_matrix(yte, rf_pred)
print(f"\n[RF] TEST ACC: {rf_acc:.3f} | F1: {rf_f1:.3f} | thr={best_t:.3f}")
print("CM [[TN FP],[FN TP]]:\n", rf_cm)

# ---------------- Logistic Regression (scaled) ----------------
scaler = StandardScaler(with_mean=True, with_std=True)
Xtrva_s = scaler.fit_transform(Xtrva_f)
Xte_s   = scaler.transform(Xte_f)

logreg = LogisticRegression(max_iter=2000, class_weight="balanced", solver="lbfgs")
logreg.fit(Xtrva_s, ytrva)
lr_pred = logreg.predict(Xte_s)
lr_acc = accuracy_score(yte, lr_pred); lr_f1 = f1_score(yte, lr_pred); lr_cm = confusion_matrix(yte, lr_pred)
print(f"\n[LogReg] TEST ACC: {lr_acc:.3f} | F1: {lr_f1:.3f}")
print("CM [[TN FP],[FN TP]]:\n", lr_cm)

# ---------------- Linear SVM (scaled) ----------------
svm = LinearSVC(class_weight="balanced", max_iter=5000)
svm.fit(Xtrva_s, ytrva)
svm_pred = (svm.decision_function(Xte_s) >= 0).astype(int)
svm_acc = accuracy_score(yte, svm_pred); svm_f1 = f1_score(yte, svm_pred); svm_cm = confusion_matrix(yte, svm_pred)
print(f"\n[LinearSVM] TEST ACC: {svm_acc:.3f} | F1: {svm_f1:.3f}")
print("CM [[TN FP],[FN TP]]:\n", svm_cm)

# (ঐচ্ছিক) ছোট রিপোর্ট
print("\n[RF] classification report:\n", classification_report(yte, rf_pred, digits=3))


Device: cuda
Total 95 | class0=48 class1=47
>> Warm-up encoder…
Ep01 | tr 0.754/0.477 | va 0.731/0.500 | best 0.500
Ep02 | tr 0.708/0.462 | va 0.683/0.500 | best 0.500
Ep03 | tr 0.704/0.492 | va 0.710/0.500 | best 0.500
Ep04 | tr 0.709/0.569 | va 0.682/0.714 | best 0.714
Ep05 | tr 0.682/0.585 | va 0.674/0.500 | best 0.714
Ep06 | tr 0.697/0.492 | va 0.691/0.500 | best 0.714
Ep07 | tr 0.694/0.492 | va 0.687/0.571 | best 0.714
Ep08 | tr 0.695/0.554 | va 0.683/0.643 | best 0.714
Ep09 | tr 0.697/0.492 | va 0.696/0.500 | best 0.714
Ep10 | tr 0.698/0.477 | va 0.677/0.643 | best 0.714
Early stop.
Image feature shapes: (65, 64) (14, 64) (16, 64)
Meta dims: 4
Final feature dims: 68
[RF] Best VAL threshold: 0.200 (F1=0.667)

[RF] TEST ACC: 0.562 | F1: 0.696 | thr=0.200
CM [[TN FP],[FN TP]]:
 [[1 7]
 [0 8]]

[LogReg] TEST ACC: 0.625 | F1: 0.625
CM [[TN FP],[FN TP]]:
 [[5 3]
 [3 5]]

[LinearSVM] TEST ACC: 0.562 | F1: 0.533
CM [[TN FP],[FN TP]]:
 [[5 3]
 [4 4]]

[RF] classification report:
         

In [ ]:
# ===== Combined Evaluation: 3D CNN | RF | Fusion RF =====
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# --------------------------------------------------------
# 1) End-to-End 3D CNN Classifier (image only)
# --------------------------------------------------------
enc.load_state_dict(torch.load("/content/enc_best.pt", map_location=device))  # trained encoder
enc.eval()
cnn_preds=[]; cnn_true=[]
with torch.no_grad():
    for x,y,_ in test_dl:
        x=x.to(device); y=y.numpy().tolist()
        logits = enc(x, feat_only=False)        # directly classifier head
        pred = logits.argmax(1).cpu().numpy().tolist()
        cnn_preds += pred; cnn_true += y

cnn_acc = accuracy_score(cnn_true, cnn_preds)
cnn_f1  = f1_score(cnn_true, cnn_preds)
cnn_cm  = confusion_matrix(cnn_true, cnn_preds)
print("\n[3D CNN] TEST ACC:", cnn_acc, "| F1:", cnn_f1)
print("CM [[TN FP],[FN TP]]:\n", cnn_cm)

# --------------------------------------------------------
# 2) RF (image feature only, no metadata)
# --------------------------------------------------------
rf_img = RandomForestClassifier(n_estimators=400, class_weight="balanced", random_state=SEED)
rf_img.fit(Xtr_img, ytr)   # only image features
rf_pred = rf_img.predict(Xte_img)
rf_acc = accuracy_score(yte, rf_pred)
rf_f1  = f1_score(yte, rf_pred)
rf_cm  = confusion_matrix(yte, rf_pred)
print("\n[RF only-img] TEST ACC:", rf_acc, "| F1:", rf_f1)
print("CM [[TN FP],[FN TP]]:\n", rf_cm)

# --------------------------------------------------------
# 3) Hybrid Fusion RF (image + metadata)
# --------------------------------------------------------
rf_fusion = RandomForestClassifier(n_estimators=800, class_weight="balanced", random_state=SEED)
rf_fusion.fit(np.vstack([Xtr_f,Xva_f]), np.hstack([ytr,yva]))
fusion_pred = rf_fusion.predict(Xte_f)
fusion_acc = accuracy_score(yte, fusion_pred)
fusion_f1  = f1_score(yte, fusion_pred)
fusion_cm  = confusion_matrix(yte, fusion_pred)
print("\n[Fusion RF] TEST ACC:", fusion_acc, "| F1:", fusion_f1)
print("CM [[TN FP],[FN TP]]:\n", fusion_cm)

# --------------------------------------------------------
# Summary
# --------------------------------------------------------
print("\n==== SUMMARY ====")
print(f"3D CNN         → ACC {cnn_acc:.3f} | F1 {cnn_f1:.3f}")
print(f"RF (img-only)  → ACC {rf_acc:.3f} | F1 {rf_f1:.3f}")
print(f"Fusion RF(img+meta) → ACC {fusion_acc:.3f} | F1 {fusion_f1:.3f}")



[3D CNN] TEST ACC: 0.625 | F1: 0.7
CM [[TN FP],[FN TP]]:
 [[3 5]
 [1 7]]

[RF only-img] TEST ACC: 0.6875 | F1: 0.7058823529411765
CM [[TN FP],[FN TP]]:
 [[5 3]
 [2 6]]

[Fusion RF] TEST ACC: 0.8125 | F1: 0.8235294117647058
CM [[TN FP],[FN TP]]:
 [[6 2]
 [1 7]]

==== SUMMARY ====
3D CNN         → ACC 0.625 | F1 0.700
RF (img-only)  → ACC 0.688 | F1 0.706
Fusion RF(img+meta) → ACC 0.812 | F1 0.824
